In [1]:
"""
思路: 在TR模型的基础上, 融合用户的文本特征, 以异质图Heter-GAT的方式融合用户特征, 做信息传播预测任务
方法:
1. 整理原始数据, 构建用户有向关联网络, 并根据原始文本内容计算用户文本嵌入向量
2. 考虑节点类型为User和Tweet, 边类型为U-U和U-T, 分别从用户特征和文本特征的角度通过GAT网络融合邻域节点特征;
   Heter-GAT模型的输出为(N,|Rs|+1,D')维度, 模型后面需要接一个全连接层FC=(|Rs|+1)*D'->2, 损失函数保持为NLL-Loss
3. 可视化局部邻域, 观察不同注意力头、不同异质图邻域卷积的偏向
"""

"\n思路: 在TR模型的基础上, 融合用户的文本特征, 以异质图Heter-GAT的方式融合用户特征, 做信息传播预测任务\n方法:\n1. 整理原始数据, 构建用户有向关联网络, 并根据原始文本内容计算用户文本嵌入向量\n2. 考虑节点类型为User和Tweet, 边类型为U-U和U-T, 分别从用户特征和文本特征的角度通过GAT网络融合邻域节点特征;\n   Heter-GAT模型的输出为(N,|Rs|+1,D')维度, 模型后面需要接一个全连接层FC=(|Rs|+1)*D'->2, 损失函数保持为NLL-Loss\n3. 可视化局部邻域, 观察不同注意力头、不同异质图邻域卷积的偏向\n"

In [2]:
# import sys
# import os
# sys.path.append(os.path.dirname(os.getcwd()))
# from lib.log import logger
# from utils import load_pickle, save_pickle, summarize_distribution, find_rt_bound, HeterSubGraphSample
# from lib.utils import get_node_types, extend_edges, create_sparse, get_sparse_tensor
# import numpy as np

In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from lib.log import logger
from lib.utils import get_sparse_tensor
from utils import HeterSubGraphSample, load_pickle, save_pickle, init_args, ChunkSampler, HeterGraphDataset, sparse_batch_collate
from model import HeterGraphAttentionNetwork
import numpy as np
import time
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, precision_recall_curve
from tensorboard_logger import tensorboard_logger
from scipy.sparse import csr_matrix


In [7]:
# NOTE: gen user feats
from utils import SubGraphSample

data_dirpath = "/root/data/HeterGAT/stages/stages_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20/"
samples = SubGraphSample(
    adj_matrices=np.load(os.path.join(data_dirpath, "adjacency_matrix.npy")),
    influence_features=np.load(os.path.join(data_dirpath, "influence_feature.npy")),
    vertex_ids=np.load(os.path.join(data_dirpath, "vertex_id.npy")),
    labels=np.load(os.path.join(data_dirpath, "label.npy")),
    tags=np.load(os.path.join(data_dirpath, "hashtag.npy")),
    time_stages=np.load(os.path.join(data_dirpath, "stage.npy"))
)
logger.info("Loading...")

inf_feats  = samples.influence_features
hashtags   = samples.tags
stages     = samples.time_stages

user_feats = []
for idx in range(len(samples)):
    if idx and idx % 1000 == 0:
        logger.info(f"Progress={idx/len(samples)}")
        break
    user_ids = samples.vertex_ids[idx]

    gra_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_gravity_feature/hashtag{hashtags[idx]}_t{stages[idx]}.p")).reshape(-1,1)
    exp_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_exptime_feature1/hashtag{hashtags[idx]}_t{stages[idx]}.p")).reshape(-1,1)
    cas_feat = np.array(load_pickle(f"/root/data/HeterGAT/user_features/norm_ce_feature/hashtag{hashtags[idx]}_t{stages[idx]}.p")).reshape(-1,1)
    user_feats.append(
        np.concatenate((inf_feats[idx], gra_feat[user_ids], exp_feat[user_ids], cas_feat[user_ids]), axis=1)
    )
user_feats = np.array(user_feats)
logger.info(f"{user_feats.shape}")
# save_pickle(user_feats, os.path.join(data_dirpath, "user_features.p"))

2022-10-22 14:05:00,177 Loading...
2022-10-22 14:06:55,587 Progress=0.016213499359566775
2022-10-22 14:06:55,591 (1000, 20, 5)


In [61]:
N, bs, n = 10, 5, 2

a = torch.rand((N, bs))
indices = torch.LongTensor([[4,5], [2,3], [6,7], [0,1], [8,9]]) # (bs, n)
# trg: (bs, n, 1)

trg = a.T.gather(dim=1, index=indices)
logger.info(trg.shape)
logger.info(trg)
logger.info(a)

2022-10-22 20:00:17,780 torch.Size([5, 2])
2022-10-22 20:00:17,782 tensor([[0.7470, 0.4498],
        [0.9738, 0.6716],
        [0.2377, 0.2962],
        [0.1450, 0.8016],
        [0.7793, 0.2376]])
2022-10-22 20:00:17,785 tensor([[0.9189, 0.4629, 0.1308, 0.1450, 0.1231],
        [0.3367, 0.6372, 0.2932, 0.8016, 0.2547],
        [0.0439, 0.9738, 0.0189, 0.4817, 0.8586],
        [0.9848, 0.6716, 0.3356, 0.9293, 0.7409],
        [0.7470, 0.2417, 0.4886, 0.5591, 0.6745],
        [0.4498, 0.4378, 0.5871, 0.5634, 0.7223],
        [0.1415, 0.3992, 0.2377, 0.5711, 0.5305],
        [0.0782, 0.6359, 0.2962, 0.8852, 0.2720],
        [0.0191, 0.8393, 0.8337, 0.0857, 0.7793],
        [0.3462, 0.4402, 0.5805, 0.5101, 0.2376]])


In [62]:
# Gen Hadjs
tweet_ids = load_pickle("/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_20_neg_1_restart_20/tweet_ids.p")


In [69]:
for idx in range(len(tweet_ids)):
    tids = tweet_ids[idx]
    for tid in tids:
        if tid >= 208894:
            continue
        logger.info("Error")
        break

2022-10-22 20:49:57,482 Error
2022-10-22 20:49:57,485 Error
2022-10-22 20:49:57,486 Error
2022-10-22 20:49:57,487 Error
2022-10-22 20:49:57,488 Error
2022-10-22 20:49:57,489 Error
2022-10-22 20:49:57,490 Error
2022-10-22 20:49:57,490 Error
2022-10-22 20:49:57,492 Error
2022-10-22 20:49:57,493 Error
2022-10-22 20:49:57,493 Error
2022-10-22 20:49:57,495 Error
2022-10-22 20:49:57,496 Error
2022-10-22 20:49:57,496 Error
2022-10-22 20:49:57,497 Error
2022-10-22 20:49:57,498 Error
2022-10-22 20:49:57,499 Error
2022-10-22 20:49:57,500 Error
2022-10-22 20:49:57,501 Error
2022-10-22 20:49:57,502 Error
2022-10-22 20:49:57,503 Error
2022-10-22 20:49:57,503 Error
2022-10-22 20:49:57,504 Error
2022-10-22 20:49:57,505 Error
2022-10-22 20:49:57,506 Error
2022-10-22 20:49:57,507 Error
2022-10-22 20:49:57,507 Error
2022-10-22 20:49:57,508 Error
2022-10-22 20:49:57,509 Error
2022-10-22 20:49:57,510 Error
2022-10-22 20:49:57,511 Error
2022-10-22 20:49:57,512 Error
2022-10-22 20:49:57,512 Error
2022-10-22

In [23]:
# def gen_user_emb(tot_user_num):
#     # 208894*200*8*3
#     user_feats = [[0.]*200*8*3 for _ in range(tot_user_num)]
#     for tag in range(200):
#         logger.info(f"tag={tag}")
#         for stage in range(8):
#             for feats_idx, feats in enumerate(["norm_gravity_feature", "norm_exptime_feature1", "norm_ce_feature"]):
#                 feats = load_pickle(f"/root/data/HeterGAT/user_features/{feats}/hashtag{tag}_t{stage}.p")
#                 for idx in range(tot_user_num):
#                     user_feats[idx][tag*3*8+stage*3+feats_idx] = float(feats[idx])
#     logger.info(f"shape={user_feats.shape}")
#     return torch.FloatTensor(user_feats)

# user_emb = gen_user_emb(208894)

# 208894*200*8*3
tot_user_num = 208894
user_feats = [[0.]*200*8*3 for _ in range(tot_user_num)]
for tag in range(200):
    logger.info(f"tag={tag}")
    for stage in range(8):
        for feats_idx, feats in enumerate(["norm_gravity_feature", "norm_exptime_feature1", "norm_ce_feature"]):
            feats = load_pickle(f"/root/data/HeterGAT/user_features/{feats}/hashtag{tag}_t{stage}.p")
            for idx in range(tot_user_num):
                user_feats[idx][tag*3*8+stage*3+feats_idx] = float(feats[idx])
user_feats = np.array(user_feats)
logger.info(f"shape={user_feats.shape}")
save_pickle(user_feats, "/root/data/HeterGAT/user_features/user_features.p")
user_emb = torch.FloatTensor(user_feats)

2022-10-22 15:00:38,171 tag=0
2022-10-22 15:00:42,130 tag=1
2022-10-22 15:00:45,979 tag=2
2022-10-22 15:00:49,769 tag=3
2022-10-22 15:00:53,529 tag=4
2022-10-22 15:00:57,284 tag=5
2022-10-22 15:01:01,043 tag=6
2022-10-22 15:01:04,816 tag=7
2022-10-22 15:01:08,604 tag=8
2022-10-22 15:01:12,377 tag=9
2022-10-22 15:01:16,140 tag=10
2022-10-22 15:01:19,967 tag=11
2022-10-22 15:01:23,934 tag=12
2022-10-22 15:01:27,878 tag=13
2022-10-22 15:02:34,273 tag=14
2022-10-22 15:02:38,360 tag=15
2022-10-22 15:02:42,337 tag=16
2022-10-22 15:02:46,308 tag=17
2022-10-22 15:02:50,373 tag=18
2022-10-22 15:02:54,344 tag=19
2022-10-22 15:02:58,307 tag=20
2022-10-22 15:03:02,269 tag=21
2022-10-22 15:03:06,204 tag=22
2022-10-22 15:03:10,134 tag=23
2022-10-22 15:03:14,114 tag=24
2022-10-22 15:03:18,049 tag=25
2022-10-22 15:03:21,998 tag=26
2022-10-22 15:03:25,954 tag=27
2022-10-22 15:03:29,903 tag=28
2022-10-22 15:03:33,855 tag=29
2022-10-22 15:03:37,908 tag=30
2022-10-22 15:03:41,983 tag=31
2022-10-22 15:03:4

In [18]:
for feats_idx, feats in enumerate(["norm_gravity_feature", "norm_exptime_feature1", "norm_ce_feature"]):
    feats = load_pickle(f"/root/data/HeterGAT/user_features/{feats}/hashtag{0}_t{0}.p")
    logger.info(feats[0])

2022-10-22 14:39:09,428 0.6372319229258053
2022-10-22 14:39:09,527 0.2223360655737705
2022-10-22 14:39:09,532 0.5099874841101689


In [11]:
feats = load_pickle("/root/data/HeterGAT/user_features/norm_gravity_feature/hashtag0_t0.p")
len(feats)

208894

In [5]:
len(samples)

61677

In [2]:
hs = load_pickle("/root/data/HeterGAT/stages/hs_subg483_inf_40_1718027_deg_18_483_ego_50_neg_1_restart_20/heter_samples.p")


In [5]:
hs.labels

(49926, 2)

In [66]:
# NOTE: Fake Digg Heter Dataset
from torch.utils.data import Dataset
from utils import SubGraphSample, load_w2v_feature

class DiggDataset(Dataset):
    def __init__(self, samples: SubGraphSample, embedding) -> None:
        super().__init__()
        self.adjs = samples.adj_matrices
        self.labels = samples.labels
        self.feats = samples.influence_features
        self.vertex_ids = samples.vertex_ids
        self.concact_feats(embedding)
    def concact_feats(self, embedding):
        feats = []
        for idx, vertex_ids in enumerate(self.vertex_ids):
            emb_feats = [embedding[user] for user in vertex_ids]
            feats.append(np.concatenate((self.feats[idx], emb_feats), axis=1))
        self.feats = np.array(feats)
        logger.info(self.feats.shape)
    def __len__(self):
        return self.labels.shape[0]
    def __getitem__(self, index):
        return self.adjs[index], self.labels[index], self.feats[index]

def collate_fn2(batch:list): 
    """
    Collate function which to transform scipy coo matrix to pytorch sparse tensor
    """
    adjs_batch, labels_batch, feats_batch = zip(*batch)
    adjs_batch = torch.FloatTensor(np.array(adjs_batch))
    
    if type(labels_batch[0]).__module__ == 'numpy':
        # NOTE: https://stackoverflow.com/questions/69742930/runtimeerror-nll-loss-forward-reduce-cuda-kernel-2d-index-not-implemented-for
        labels_batch = torch.LongTensor(labels_batch)
    
    if type(feats_batch[0]).__module__ == 'numpy':
        feats_batch = torch.FloatTensor(np.array(feats_batch))
    return adjs_batch, labels_batch, feats_batch

def digg_load_dataset(train_ratio=60, valid_ratio=20, batch_size=256):
    embedding_path = "/root/Lab_Related/data/Heter-GAT/Classic/deepwalk/deepwalk_added.emb_64"
    vertices = np.load("/root/TR-pptusn/DeepInf-preprocess/preprocess/stages_op_inf_100_1k/vertex_id.npy")
    max_vertex_idx = np.max(vertices)
    embedding = load_w2v_feature(embedding_path, max_vertex_idx)
    # embedding = torch.FloatTensor(embedding)

    samples = SubGraphSample(
        adj_matrices=np.load("/root/TR-pptusn/DeepInf-preprocess/preprocess/stages_op_inf_100_1k/adjacency_matrix.npy"),
        influence_features=np.load("/root/TR-pptusn/DeepInf-preprocess/preprocess/stages_op_inf_100_1k/influence_feature.npy"),
        vertex_ids=np.load("/root/TR-pptusn/DeepInf-preprocess/preprocess/stages_op_inf_100_1k/vertex_id.npy"),
        labels=np.load("/root/TR-pptusn/DeepInf-preprocess/preprocess/stages_op_inf_100_1k/label.npy")
    )
    dataset = DiggDataset(samples, embedding)
    nb_samples    = len(dataset)
    
    train_start,  valid_start, test_start = 0, int(nb_samples*train_ratio/100), int(nb_samples*(train_ratio+valid_ratio)/100)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(valid_start-train_start, 0), collate_fn=collate_fn2)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(test_start-valid_start, valid_start), collate_fn=collate_fn2)
    test_loader  = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(nb_samples - test_start, test_start), collate_fn=collate_fn2)
    logger.info(f"Finish Loading Dataset... train={len(train_loader)}, valid={len(valid_loader)}, test={len(test_loader)}")

    return samples, train_loader, valid_loader, test_loader

In [67]:
GPU_MODEL = 'cuda:2'

# 1. 
def load_dataset(data_filepath:str, train_ratio:float, valid_ratio:float, batch_size:int):
    # heter_samples = load_pickle(os.path.join(data_dirpath, "heter_samples_tensor.p"))
    heter_samples = load_pickle(data_filepath)
    dataset       = HeterGraphDataset(heter_samples=heter_samples)
    nb_samples    = len(dataset)
    
    train_start,  valid_start, test_start = 0, int(nb_samples*train_ratio/100), int(nb_samples*(train_ratio+valid_ratio)/100)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(valid_start-train_start, 0), collate_fn=sparse_batch_collate)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(test_start-valid_start, valid_start), collate_fn=sparse_batch_collate)
    test_loader  = DataLoader(dataset, batch_size=batch_size, sampler=ChunkSampler(nb_samples - test_start, test_start), collate_fn=sparse_batch_collate)
    logger.info(f"Finish Loading Dataset... train={len(train_loader)}, valid={len(valid_loader)}, test={len(test_loader)}")

    return heter_samples, train_loader, valid_loader, test_loader

# args = init_args()
args = {
    "train_ratio": 60,
    "valid_ratio": 20,
    "batch": 256,
    # "nb_user": 50,
    "class_weight_balanced": True,
    "hidden_units": "16,16",
    "heads": "8,8",
    "cuda": True,
    "lr": 0.1,
    "weight_decay": 5e-4,
    "dropout": 0.2,
    "seed": 42,
    "epochs": 100,
    "check_point": 2,
    # "file_dir": "heter_samples_ratio1/0/heter_samples.p",
    "file_dir": "hs_new_ratio1/heter_samples.p",
}
# heter_samples, train_loader, valid_loader, test_loader = load_dataset(args["file_dir"], args["train_ratio"], args["valid_ratio"], args["batch"])
samples, train_loader, valid_loader, test_loader = digg_load_dataset()
nb_samples = len(samples)
nb_classes = 2
class_weight = torch.FloatTensor(nb_samples / (nb_classes*np.bincount(samples.labels))) if args["class_weight_balanced"] else torch.ones(nb_classes)
nb_user = 50
n_units = [samples.influence_features.shape[2]+64]+[int(x) for x in args["hidden_units"].strip().split(",")]
n_heads = [int(x) for x in args["heads"].strip().split(",")]
logger.info(f"nb_samples={nb_samples}, class_weight={class_weight[0]:.2f}:{class_weight[1]:.2f}, n_units={n_units}, n_heads={n_heads}")
# logger.info("class_weight=%.2f:%.2f", class_weight[0], class_weight[1])

2022-10-12 15:42:11,284 n=204955, d=64
2022-10-12 15:42:21,005 (38152, 50, 66)
2022-10-12 15:42:21,017 Finish Loading Dataset... train=90, valid=30, test=30
2022-10-12 15:42:21,184 nb_samples=38152, class_weight=0.57:4.05, n_units=[66, 16, 16], n_heads=[8, 8]


In [68]:
# 2. 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.parameter import Parameter
import copy
from model import BatchSparseMultiHeadGraphAttention, BatchMultiHeadGraphAttention, BatchAdditiveAttention

class HeterGraphAttentionNetwork2(nn.Module):
    def __init__(
        self, n_user, nb_node_kinds=2, nb_loop_nodes=[50,1050],
        nb_classes=2, n_units=[25,64], n_heads=[3],
        attn_dropout=0.5, dropout=0.1, 
        d2=64, gpu_device_ids=[] 
    ) -> None:
        """
        Args:
            gpu_device_ids(List[int], default=[]): 采用Model Parallel方法, 主要使多头注意力可以在不同GPU上运行, 
                模型以数据所属GPU为例, 先在该参数指定的不同GPU上执行单一注意力头, 再在最后将所有注意力头的运行结果复制回数据所属的主GPU上
        """
        super().__init__()

        self.n_layer = len(n_units) - 1
        self.dropout = dropout
        self.gpu_device_ids = gpu_device_ids

        self.d = n_units[0]
        self.d1 = n_units[1]
        self.d2 = n_units[1]
        self.n_user = n_user

        self.layer_stack = nn.ModuleList()
        for hidx in range(nb_node_kinds):
            layer_stack = nn.ModuleList()
            for i in range(self.n_layer):
                # consider multi head from last layer
                f_in = n_units[i] * n_heads[i - 1] if i else n_units[i]
                layer_stack.append(
                    # BatchMultiHeadGraphAttention(nb_heads=n_heads[i], nb_in_feats=f_in, nb_out_feats=n_units[i + 1], 
                    #     nb_loop_nodes=nb_loop_nodes[hidx], attn_dropout=attn_dropout)
                    BatchMultiHeadGraphAttention(n_head=n_heads[i], f_in=f_in, f_out=n_units[i+1], attn_dropout=attn_dropout)
                )
            self.layer_stack.append(layer_stack)
        self.additive_attention = BatchAdditiveAttention(d=self.d, d1=self.d1, d2=self.d2)
        self.fc_layer = nn.Linear(in_features=self.d1*(nb_node_kinds+1), out_features=nb_classes)
    
    def forward(self, h, hadj):
        # NOTE: h: (bs, N, fin), hadj: (|Rs|, bs, N, N)
        bs, n = h.shape[:2]
        heter_embs = []
        for heter_idx, layer_stack in enumerate(self.layer_stack):
            x = copy.deepcopy(h)
            for i, gat_layer in enumerate(layer_stack):
                x = gat_layer(x, hadj[heter_idx]) # output: (bs, n_head, n, f_out)
                if i + 1 == self.n_layer:
                    x = x.mean(dim=-3) # (bs, n_head, n, f_out) -> (bs, n, f_out)
                else:
                    x = F.elu(x.reshape(bs, n, -1))
                    x = F.dropout(x, self.dropout, training=self.training)
            heter_embs.append(x[:,:self.n_user].unsqueeze(-2)) # (bs, Nu, 1, f_out)
        type_aware_emb = torch.cat(heter_embs, dim=-2) # (bs, Nu, |Rs|, D')
        type_fusion_emb = self.additive_attention(h[:,:self.n_user], type_aware_emb) # (bs, Nu, 1, D')
        ret = self.fc_layer(
            torch.cat((type_fusion_emb, type_aware_emb), dim=-2).reshape(bs, self.n_user,-1) # (bs, Nu, |Rs|+1, D') -> (bs, Nu, (|Rs|+1)*D')
        ) #  (bs, Nu, nb_classes)
        return F.log_softmax(ret, dim=-1)

model = HeterGraphAttentionNetwork2(n_user=nb_user, n_units=n_units, nb_classes=nb_classes, n_heads=n_heads, dropout=args["dropout"])
if args["cuda"]:
    # model.cuda()
    model.to(GPU_MODEL)
    # class_weight = class_weight.cuda()
    class_weight = class_weight.to(GPU_MODEL)
# params = [{'params': model.parameters()}]
optimizer = optim.Adagrad(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

In [69]:
def train(epoch, train_loader, valid_loader, test_loader, log_desc='train_'):
    model.train()

    loss = 0.
    total = 0.
    for i_batch, batch in enumerate(train_loader):
        # if i_batch % 10 == 0:
        #     logger.info(f"i_batch={i_batch}")
        adjs, labels, feats = batch
        bs = adjs.size(0)

        if args["cuda"]:
            adjs    = adjs.to(GPU_MODEL)
            labels  = labels.to(GPU_MODEL)
            feats   = feats.to(GPU_MODEL)

        optimizer.zero_grad()
        output = model(feats, torch.stack([adjs, adjs]))
        output = output[:,-1,:] # choose last user

        loss_train = F.nll_loss(output, labels, class_weight)
        loss += bs * loss_train.item()
        total += bs
        loss_train.backward()
        optimizer.step()
    logger.info("train loss in this epoch %f", loss / total)

train(0, train_loader, valid_loader, test_loader)

2022-10-12 15:46:18,982 train loss in this epoch 1.182631


In [ ]:
# for i_batch, batch in enumerate(train_loader):
#     uu_adjs, ut_adjs, labels, feats = batch
#     bs = uu_adjs.size(0)

#     # feats = feats[:,:100]
#     # labels = labels
#     # nw_uu_adjs, nw_ut_adjs = [], []
#     # for idx in range(len(uu_adjs)):
#     #     nw_uu_adjs.append(torch.LongTensor(uu_adjs[idx].to_dense().numpy()[:100,:100]+np.eye(100)))
#     #     nw_ut_adjs.append(torch.LongTensor(ut_adjs[idx].to_dense().numpy()[:100,:100]+np.eye(100)))
#     # uu_adjs = torch.stack(nw_uu_adjs)
#     # ut_adjs = torch.stack(nw_ut_adjs)

#     if args["cuda"]:
#         uu_adjs, ut_adjs, labels, feats = uu_adjs.to(GPU_MODEL), ut_adjs.to(GPU_MODEL), labels.to(GPU_MODEL), feats.to(GPU_MODEL)
#     break

# model = HeterGraphAttentionNetwork(n_user=nb_user, n_units=n_units, nb_classes=nb_classes, n_heads=n_heads, dropout=args["dropout"])
# if args["cuda"]:
#     model.to(GPU_MODEL)
#     class_weight = class_weight.to(GPU_MODEL)
# optimizer = optim.Adagrad(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

# output = model(feats, torch.stack([uu_adjs, ut_adjs]))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.parameter import Parameter
from model import BatchSparseMultiHeadGraphAttention, BatchAdditiveAttention, BatchMultiHeadGraphAttention
import copy

class HeterGraphAttentionNetwork2(nn.Module):
    def __init__(
        self, n_user, nb_node_kinds=2, nb_loop_nodes=[50,1050],
        nb_classes=2, n_units=[25,64], n_heads=[3],
        attn_dropout=0.5, dropout=0.1, 
        d2=64, gpu_device_ids=[] 
    ) -> None:
        super().__init__()

        self.n_layer = len(n_units) - 1
        self.dropout = dropout
        self.gpu_device_ids = gpu_device_ids

        self.d = n_units[0]
        self.d1 = n_units[1]
        self.d2 = d2
        self.n_user = n_user

        self.layer_stack = nn.ModuleList()
        for hidx in range(nb_node_kinds):
            layer_stack = nn.ModuleList()
            for i in range(self.n_layer):
                # consider multi head from last layer
                f_in = n_units[i] * n_heads[i - 1] if i else n_units[i]
                layer_stack.append(
                    BatchMultiHeadGraphAttention(n_head=n_heads[i], f_in=f_in, f_out=n_units[i + 1], 
                        attn_dropout=attn_dropout)
                )
            self.layer_stack.append(layer_stack)
        self.additive_attention = BatchAdditiveAttention(d=self.d, d1=self.d1, d2=self.d2)
        self.fc_layer = nn.Linear(in_features=self.d1*(nb_node_kinds+1), out_features=nb_classes)
    
    def forward(self, h, hadj):
        # NOTE: h: (bs, N, fin), hadj: (|Rs|, bs, N, N)
        bs, n = h.shape[:2]
        heter_embs = []
        for heter_idx, layer_stack in enumerate(self.layer_stack):
            x = copy.deepcopy(h)
            for i, gat_layer in enumerate(layer_stack):
                x = gat_layer(x, hadj[heter_idx]) # output: (bs, n_head, n, f_out)
                if i + 1 == self.n_layer:
                    x = x.mean(dim=-3) # (bs, n_head, n, f_out) -> (bs, n, f_out)
                else:
                    x = F.elu(x.reshape(bs, n, -1))
                    x = F.dropout(x, self.dropout, training=self.training)
            logger.info(x)
            heter_embs.append(x[:,:self.n_user].unsqueeze(-2)) # (bs, Nu, 1, f_out)
        type_aware_emb = torch.cat(heter_embs, dim=-2) # (bs, Nu, |Rs|, D')
        type_fusion_emb = self.additive_attention(h[:,:self.n_user], type_aware_emb) # (bs, Nu, 1, D')
        ret = self.fc_layer(
            torch.cat((type_fusion_emb, type_aware_emb), dim=-2).reshape(bs, self.n_user,-1) # (bs, Nu, |Rs|+1, D') -> (bs, Nu, (|Rs|+1)*D')
        ) #  (bs, Nu, nb_classes)
        return F.log_softmax(ret, dim=-1)

model = HeterGraphAttentionNetwork2(n_user=nb_user, n_units=n_units, nb_classes=nb_classes, n_heads=n_heads, dropout=args["dropout"])
if args["cuda"]:
    model.to(GPU_MODEL)
    class_weight = class_weight.to(GPU_MODEL)
optimizer = optim.Adagrad(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

output = model(feats, torch.stack([uu_adjs, ut_adjs]))

In [8]:
# 2. 
model = HeterGraphAttentionNetwork(n_user=nb_user, n_units=n_units, nb_classes=nb_classes, n_heads=n_heads, dropout=args["dropout"])
if args["cuda"]:
    # model.cuda()
    model.to(GPU_MODEL)
    # class_weight = class_weight.cuda()
    class_weight = class_weight.to(GPU_MODEL)
# params = [{'params': model.parameters()}]
optimizer = optim.Adagrad(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])


In [4]:
# logger.info(f"out Allocated: {torch.cuda.memory_reserved(int(GPU_MODEL[-1]))/1024**3}")

2022-09-22 12:14:19,308 out Allocated: 0.001953125


In [11]:
def evaluate(epoch, loader, thr=None, return_best_thr=False, log_desc='valid_'):
    model.eval()
    total = 0.
    loss, prec, rec, f1 = 0., 0., 0., 0.
    y_true, y_pred, y_score = [], [], []
    for i_batch, batch in enumerate(loader):
        uu_adjs, ut_adjs, labels, feats = batch
        bs = uu_adjs.size(0)

        if args["cuda"]:
            uu_adjs = uu_adjs.to(GPU_MODEL)
            ut_adjs = ut_adjs.to(GPU_MODEL)
            labels  = labels.to(GPU_MODEL)
            feats   = feats.to(GPU_MODEL)

        output = model(feats, torch.stack([uu_adjs, ut_adjs]))
        output = output[:,-1,:] # choose last user

        loss_batch = F.nll_loss(output, labels, class_weight)
        loss += bs * loss_batch.item()

        y_true += labels.data.tolist()
        # 返回output中每行最大值的索引
        y_pred += output.max(1)[1].data.tolist()
        y_score += output[:, 1].data.tolist()
        total += bs

    model.train()

    if thr is not None:
        logger.info("using threshold %.4f", thr)
        y_score = np.array(y_score)
        y_pred = np.zeros_like(y_score)
        y_pred[y_score > thr] = 1

    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary")
    auc = roc_auc_score(y_true, y_score)
    logger.info("%sloss: %.4f AUC: %.4f Prec: %.4f Rec: %.4f F1: %.4f",
            log_desc, loss / total, auc, prec, rec, f1)

    if return_best_thr:
        precs, recs, thrs = precision_recall_curve(y_true, y_score)
        f1s = 2 * precs * recs / (precs + recs)
        f1s = f1s[:-1]
        thrs = thrs[~np.isnan(f1s)]
        f1s = f1s[~np.isnan(f1s)]
        best_thr = thrs[np.argmax(f1s)]
        logger.info("best threshold=%4f, f1=%.4f", best_thr, np.max(f1s))
        return best_thr
    else:
        return None

def train(epoch, train_loader, valid_loader, test_loader, log_desc='train_'):
    model.train()

    loss = 0.
    total = 0.
    for i_batch, batch in enumerate(train_loader):
        # if i_batch % 10 == 0:
        #     logger.info(f"i_batch={i_batch}")
        uu_adjs, ut_adjs, labels, feats = batch
        bs = uu_adjs.size(0)

        if args["cuda"]:
            uu_adjs = uu_adjs.to(GPU_MODEL)
            ut_adjs = ut_adjs.to(GPU_MODEL)
            labels  = labels.to(GPU_MODEL)
            feats   = feats.to(GPU_MODEL)

        optimizer.zero_grad()
        output = model(torch.rand((feats.shape)).to(GPU_MODEL), torch.stack([uu_adjs, ut_adjs]))
        logger.info(output.shape)
        output = output[:,-1,:] # choose last user

        loss_train = F.nll_loss(output, labels, class_weight)
        loss += bs * loss_train.item()
        total += bs
        loss_train.backward()
        optimizer.step()
    logger.info("train loss in this epoch %f", loss / total)
    if (epoch + 1) % args["check_point"] == 0:
        logger.info("epoch %d, checkpoint!", epoch)
        best_thr = evaluate(epoch, valid_loader, return_best_thr=True, log_desc='valid_')
        evaluate(epoch, test_loader, thr=best_thr, log_desc='test_')

# for epoch in range(args["epochs"]):
#     train(epoch, train_loader, valid_loader, test_loader)

best_thr = evaluate(0, valid_loader, return_best_thr=True, log_desc='valid_')
evaluate(0, test_loader, thr=best_thr, log_desc='test_')
"""
2022-09-19 18:36:01,818 train loss in this epoch 0.304292
2022-09-19 18:45:58,481 train loss in this epoch 0.096567
2022-09-19 18:45:58,484 epoch 1, checkpoint!
2022-09-19 18:47:17,276 valid_loss: 0.0495 AUC: 0.9972 Prec: 0.9972 Rec: 1.0000 F1: 0.9986
/tmp/ipykernel_30944/2709401001.py:55: RuntimeWarning: invalid value encountered in true_divide
  f1s = 2 * precs * recs / (precs + recs)
2022-09-19 18:47:17,301 best threshold=-0.040288, f1=0.9990
2022-09-19 18:48:35,552 using threshold -0.0403
2022-09-19 18:48:35,601 test_loss: 0.0501 AUC: 0.9976 Prec: 0.9975 Rec: 1.0000 F1: 0.9988
"""

2022-09-22 12:42:28,618 torch.Size([256, 50, 2])
2022-09-22 12:42:28,620 torch.Size([256, 2])
2022-09-22 12:42:29,528 torch.Size([256, 50, 2])
2022-09-22 12:42:29,529 torch.Size([256, 2])
2022-09-22 12:42:30,180 torch.Size([256, 50, 2])
2022-09-22 12:42:30,181 torch.Size([256, 2])


KeyboardInterrupt: 